In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

2025-10-13 15:22:59.117719: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-13 15:22:59.193419: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-13 15:23:01.270277: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
df = pd.read_csv('sensor_dataset.csv')

In [3]:
X = df[['mq2', 'temperature', 'humidity']].values
y = df['label_id'].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [6]:
model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(3,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(4, activation='softmax')  # 4 classes
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

/home/mj/.local/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-10-13 15:23:02.325714: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
history = model.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6070 - loss: 1.3600 - val_accuracy: 0.5833 - val_loss: 1.2481
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6439 - loss: 1.1675 - val_accuracy: 0.6288 - val_loss: 1.0395
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6866 - loss: 0.9622 - val_accuracy: 0.7083 - val_loss: 0.8403
Epoch 4/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7273 - loss: 0.8017 - val_accuracy: 0.7348 - val_loss: 0.6792
Epoch 5/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7509 - loss: 0.6552 - val_accuracy: 0.7879 - val_loss: 0.5587
Epoch 6/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7727 - loss: 0.5775 - val_accuracy: 0.8485 - val_loss: 0.4686
Epoch 7/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8134 - loss: 0.5218 - val_accuracy: 0.8674 - val_loss: 0.3995
Epoch 8/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8400 - loss: 0.4401 - val_accuracy: 0.8712 - val_loss

In [8]:
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\nTest accuracy: {test_acc:.4f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9909 - loss: 0.0376 

Test accuracy: 0.9909


In [9]:
# model.save('sensor_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('sensor_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpnmmxn2u8/assets


INFO:tensorflow:Assets written to: /tmp/tmpnmmxn2u8/assets


Saved artifact at '/tmp/tmpnmmxn2u8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  139954099622864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139954099624208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139954099621904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139954099623824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139954099624016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139954099624592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139954099624400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139954099624976: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1760361793.380391  187647 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1760361793.380417  187647 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-10-13 15:23:13.380842: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpnmmxn2u8
2025-10-13 15:23:13.381545: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-10-13 15:23:13.381562: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpnmmxn2u8
I0000 00:00:1760361793.387498  187647 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled
2025-10-13 15:23:13.389016: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-10-13 15:23:13.426776: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpnmmxn2u8
2025-10-13 15:23:13.438281: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 

In [10]:
label_map = {0: 'normal', 1: 'gas_leak', 2: 'fire', 3: 'high_humidity'}
print("\nTest predictions:")
test_samples = X_test_scaled[:5]
predictions = model.predict(test_samples)
for i, pred in enumerate(predictions):
    pred_class = np.argmax(pred)
    actual_class = y_test[i]
    print(f"Sample {i+1}: Predicted={label_map[pred_class]} ({pred[pred_class]:.2f}), Actual={label_map[actual_class]}")



Test predictions:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Sample 1: Predicted=high_humidity (0.98), Actual=high_humidity
Sample 2: Predicted=normal (1.00), Actual=normal
Sample 3: Predicted=normal (1.00), Actual=normal
Sample 4: Predicted=normal (1.00), Actual=normal
Sample 5: Predicted=normal (1.00), Actual=normal
